<a href="https://colab.research.google.com/github/sabyasm/ipython-notebooks/blob/master/Pursuit_of_Happiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U mlens

    100% |████████████████████████████████| 235kB 7.6MB/s 


In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.metrics import accuracy_score
import xgboost as xgb

from mlens.metrics import make_scorer
accuracy_scorer = make_scorer(accuracy_score, greater_is_better=True)


from mlens.ensemble import SuperLearner
from mlens.model_selection import Evaluator


# A host of Scikit-learn models
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.kernel_approximation import Nystroem
from sklearn.kernel_approximation import RBFSampler
from sklearn.pipeline import make_pipeline
from scipy.stats import randint


[MLENS] backend: threading


# **Download Feature Data stored in pickle format**

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1KvpsAYprIzDISjY3pIltj8rTjhkBmq5C'})
download.GetContentFile('features.pkl')

#https://drive.google.com/open?id=1KvpsAYprIzDISjY3pIltj8rTjhkBmq5C

In [0]:
# Assuming pickle feats are already downloaded
import pickle
file = open('features.pkl', 'rb')
labels = [[],[]]
train_features = pickle.load(file)
test_features = pickle.load(file)
labels[0] = pickle.load(file)
labels[1] = pickle.load(file)
uid = pickle.load(file)
file.close()

In [0]:
xtrain, xtest, ytrain, ytest = train_features, test_features, np.array(labels[0]), np.array(labels[1])

In [0]:
#0.77 :'(

seed = 2017
np.random.seed(seed)
ensemble = SuperLearner(scorer=accuracy_score, random_state=seed,verbose=2)

# Build the first layer
ensemble.add([RandomForestClassifier(random_state=seed), LogisticRegression(C=1),GradientBoostingClassifier(n_estimators=100, random_state=seed),MLPClassifier((80, 10), early_stopping=False, random_state=seed),GaussianNB()])

# Build the second layer
ensemble.add([LogisticRegression(), SVC(),xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)])

# Attach the final meta estimator
ensemble.add_meta(RandomForestClassifier(random_state=seed))

# --- Use ---

# Fit ensemble
ensemble.fit(xtrain, ytrain)

# Predict
preds = ensemble.predict(xtest)


Fitting 3 layers
Processing layer-1             done | 00:43:21
Processing layer-2             done | 00:00:01
Processing layer-3             done | 00:00:00
Fit complete                        | 00:43:22

Predicting 3 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00
Predict complete                    | 00:00:01




```
Fit data:
                                       score-m  score-s    ft-m   ft-s  pt-m  pt-s
layer-1  gaussiannb                       0.70     0.00    1.55   0.16  1.03  0.10
layer-1  gradientboostingclassifier       0.76     0.01  998.43   1.68  0.18  0.00
layer-1  logisticregression               0.77     0.00   17.32   0.01  0.12  0.00
layer-1  mlpclassifier                    0.76     0.00  213.45  38.07  0.51  0.14
layer-1  randomforestclassifier           0.70     0.00    6.49   0.01  0.20  0.01
layer-2  logisticregression               0.77     0.01    0.00   0.00  0.00  0.00
layer-2  svc                              0.77     0.00    0.17   0.00  0.14  0.01
layer-2  xgbclassifier                    0.77     0.00    0.17   0.03  0.01  0.00
```




In [28]:
# 01 - ensemble
import xgboost as xgb
seed = 2017
np.random.seed(seed)
ensemble = SuperLearner(scorer=accuracy_score, random_state=seed,verbose=2)

# Build the first layer
ensemble.add([
              RandomForestClassifier(random_state=seed,n_jobs=-1), 
              RandomForestClassifier(random_state=seed,n_estimators=100,n_jobs=-1), #0.74
              LogisticRegression(C=1), #.77 ~ 0.759
              #GradientBoostingClassifier(n_estimators=100, random_state=seed),
              MLPClassifier((256, 256, 64,8), early_stopping=True, random_state=seed,verbose=True), #0.76 overall 74.5
              MLPClassifier((100, 10), early_stopping=True, random_state=seed,verbose=True) ,#only 0.75 overall 0.754
              GaussianNB() #0.704
              #xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7) #28 min 76 acuracy
             ])

# Build the second layer
ensemble.add([
              LogisticRegression(),
              SVC(),
              xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)
              ])
      
# Attach the final meta estimator
ensemble.add_meta(RandomForestClassifier(random_state=seed))

# --- Use ---

# Fit ensemble
ensemble.fit(xtrain, ytrain)

# Predict
preds = ensemble.predict(xtest)

print("Fit data:\n%r" % ensemble.data)
print("Prediction score: %.3f" % accuracy_score(preds, ytest))


Fitting 3 layers
Processing layer-1             Iteration 1, loss = 0.62138745
Validation score: 0.758621
Iteration 1, loss = 0.58180038
Validation score: 0.737069
Iteration 2, loss = 0.54419475
Validation score: 0.762931
Iteration 3, loss = 0.46816738
Validation score: 0.750000
Iteration 2, loss = 0.48609276
Validation score: 0.788793
Iteration 4, loss = 0.41903247
Validation score: 0.754310
Iteration 5, loss = 0.35800397
Validation score: 0.737069
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 3, loss = 0.42604524
Validation score: 0.778017
Iteration 1, loss = 0.62204598
Validation score: 0.724138
Iteration 2, loss = 0.53052635
Validation score: 0.767241
Iteration 4, loss = 0.37323962
Validation score: 0.767241
Iteration 3, loss = 0.47464669
Validation score: 0.771552
Iteration 4, loss = 0.44177533
Validation score: 0.788793
Iteration 5, loss = 0.30374164
Validation score: 0.721983
Validation score did not improve more than to

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


done | 00:00:01
Processing layer-3             done | 00:00:00
Fit complete                        | 00:05:11

Predicting 3 layers
Processing layer-1             done | 00:00:01
Processing layer-2             done | 00:00:00
Processing layer-3             done | 00:00:00


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Predict complete                    | 00:00:01
Fit data:
                                     score-m  score-s   ft-m   ft-s  pt-m  pt-s
layer-1  gaussiannb                     0.70     0.00   1.22   0.32  0.89  0.07
layer-1  logisticregression             0.77     0.00  18.54   0.12  0.11  0.00
layer-1  mlpclassifier-1                0.76     0.01  59.00  12.47  1.66  0.01
layer-1  mlpclassifier-2                0.75     0.02  34.19  14.28  0.76  0.03
layer-1  randomforestclassifier-1       0.70     0.00   4.13   0.00  0.27  0.06
layer-1  randomforestclassifier-2       0.74     0.01  39.99   0.26  0.42  0.01
layer-2  logisticregression             0.77     0.00   0.01   0.00  0.00  0.00
layer-2  svc                            0.75     0.02   0.21   0.00  0.13  0.00
layer-2  xgbclassifier                  0.75     0.00   0.21   0.03  0.01  0.00

Prediction score: 0.772


In [0]:
print("Prediction score: %.3f" % accuracy_score(preds, ytest))

Prediction score: 0.780


In [0]:
model_name="ensemble_0783"
ts = pd.DataFrame(
{'Unique ID': uid,
 'label': preds
})
ts.to_csv(model_name+".csv",index=False)

In [0]:
from google.colab import files
files.download(model_name+".csv") 

In [0]:
#added deep layer to see if its better than 0.783
seed = 2017
np.random.seed(seed)

from mlens.ensemble import SequentialEnsemble

ensemble = SequentialEnsemble()

# The initial layer is a blended layer, same as a layer in the BlendEnsemble
ensemble.add('blend',
             [LogisticRegression(C=1), RandomForestClassifier(random_state=seed, n_jobs=-1),MLPClassifier((80, 10), early_stopping=False, random_state=seed),GradientBoostingClassifier(n_estimators=100, random_state=seed),
             MLPClassifier((256, 256, 64,8), early_stopping=True, random_state=seed,verbose=True)])

# The second layer is a stacked layer, same as a layer of the SuperLearner
ensemble.add('stack', 
             [LogisticRegression(C=1), RandomForestClassifier(random_state=seed, n_jobs=-1),MLPClassifier((80, 10), early_stopping=False, random_state=seed),GradientBoostingClassifier(n_estimators=100, random_state=seed)])

# The third layer is a subsembled layer, same as a layer of the Subsemble
ensemble.add('subsemble', [LogisticRegression(C=1), SVC(),RandomForestClassifier(random_state=seed), xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7),GaussianNB()])

# The meta estimator is added as in any other ensemble
ensemble.add_meta(MLPClassifier((80, 10), early_stopping=False, random_state=seed))

# Fit ensemble
ensemble.fit(xtrain, ytrain)

# Predict
preds = ensemble.predict(xtest)

In [0]:
#0.783
from mlens.ensemble import SequentialEnsemble

ensemble = SequentialEnsemble()

# The initial layer is a blended layer, same as a layer in the BlendEnsemble
ensemble.add('blend',
             [LogisticRegression(C=1), RandomForestClassifier(random_state=seed, n_jobs=-1),MLPClassifier((80, 10), early_stopping=False, random_state=seed),GradientBoostingClassifier(n_estimators=100, random_state=seed)])

# The second layer is a stacked layer, same as a layer of the SuperLearner
ensemble.add('stack', 
             [LogisticRegression(C=1), RandomForestClassifier(random_state=seed, n_jobs=-1),MLPClassifier((80, 10), early_stopping=False, random_state=seed),GradientBoostingClassifier(n_estimators=100, random_state=seed)])

# The third layer is a subsembled layer, same as a layer of the Subsemble
ensemble.add('subsemble', [LogisticRegression(C=1), SVC(), xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7),GaussianNB(),MLPClassifier((80, 10), early_stopping=False, random_state=seed)])

# The meta estimator is added as in any other ensemble
ensemble.add_meta(RandomForestClassifier(random_state=seed))

# Fit ensemble
ensemble.fit(xtrain, ytrain)

# Predict
preds = ensemble.predict(xtest)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

In [12]:
#0.783 -produced twice
from mlens.ensemble import SequentialEnsemble

ensemble = SequentialEnsemble()

# The initial layer is a blended layer, same as a layer in the BlendEnsemble
ensemble.add('blend',
             [LogisticRegression(C=1), RandomForestClassifier(random_state=seed, n_jobs=-1),MLPClassifier((80, 10), early_stopping=False, random_state=seed),
             GradientBoostingClassifier(n_estimators=100, random_state=seed)
             ])

# The second layer is a stacked layer, same as a layer of the SuperLearner
ensemble.add('stack', 
             [LogisticRegression(C=1), RandomForestClassifier(random_state=seed, n_jobs=-1),MLPClassifier((80, 10), early_stopping=False, random_state=seed)])

# The third layer is a subsembled layer, same as a layer of the Subsemble
ensemble.add('subsemble', [SVC(), xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7),GaussianNB()])

# The meta estimator is added as in any other ensemble
ensemble.add_meta(RandomForestClassifier(random_state=seed))

# Fit ensemble
ensemble.fit(xtrain, ytrain)

# Predict
preds = ensemble.predict(xtest)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

In [13]:
print("Prediction score: %.3f" % accuracy_score(preds, ytest))

Prediction score: 0.783


In [14]:
print("Fit data:\n%r" % ensemble.data)

Fit data:
                                             ft-m  ft-s  pt-m  pt-s
layer-1  gradientboostingclassifier  0    1302.43  0.00  0.18  0.00
layer-1  logisticregression          0      19.00  0.00  0.10  0.00
layer-1  mlpclassifier               0     164.79  0.00  0.67  0.00
layer-1  randomforestclassifier      0       4.15  0.00  0.25  0.00
layer-2  logisticregression          0       0.00  0.00  0.00  0.00
layer-2  mlpclassifier               0       1.04  0.38  0.00  0.00
layer-2  randomforestclassifier      0       0.12  0.00  0.10  0.00
layer-3  gaussiannb                  0       0.00  0.00  0.00  0.00
layer-3  gaussiannb                  1       0.00  0.00  0.00  0.00
layer-3  svc                         0       0.01  0.00  0.02  0.01
layer-3  svc                         1       0.01  0.00  0.01  0.00
layer-3  xgbclassifier               0       0.03  0.00  0.00  0.00
layer-3  xgbclassifier               1       0.03  0.00  0.00  0.00



In [0]:
# giant one
from mlens.ensemble import SequentialEnsemble

ensemble = SequentialEnsemble()


# The initial layer is a blended layer, same as a layer in the BlendEnsemble
ensemble.add('blend', [
              RandomForestClassifier(random_state=seed,n_jobs=-1), 
              RandomForestClassifier(random_state=seed,n_estimators=100,n_jobs=-1), #0.74
              LogisticRegression(C=1), #.77 ~ 0.759
              MLPClassifier((256, 256, 64,8), early_stopping=True, random_state=seed,verbose=True), #0.76 overall 74.5
              GradientBoostingClassifier(n_estimators=100, random_state=seed),
              MLPClassifier((256, 256, 64,8), early_stopping=True, random_state=seed,verbose=True), #0.76 overall 74.5
              MLPClassifier((100, 10), early_stopping=True, random_state=seed,verbose=True) ,#only 0.75 overall 0.754
              GaussianNB(), #0.704
              xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7) #28 min 76 acuracy
])

# The second layer is a stacked layer, same as a layer of the SuperLearner
ensemble.add('stack', 
             [
              RandomForestClassifier(random_state=seed,n_jobs=-1), 
              RandomForestClassifier(random_state=seed,n_estimators=100,n_jobs=-1), #0.74
              LogisticRegression(C=1), #.77 ~ 0.759
              GradientBoostingClassifier(n_estimators=100, random_state=seed),
              MLPClassifier((256, 256, 64,8), early_stopping=True, random_state=seed,verbose=True), #0.76 overall 74.5
              MLPClassifier((100, 10), early_stopping=True, random_state=seed,verbose=True) ,#only 0.75 overall 0.754
              GaussianNB(), #0.704
              xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7) #28 min 76 acuracy
             ])

# The third layer is a subsembled layer, same as a layer of the Subsemble
ensemble.add('subsemble', [SVC(), xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7)])

# The meta estimator is added as in any other ensemble
ensemble.add_meta(RandomForestClassifier(random_state=seed))

# Fit ensemble
ensemble.fit(xtrain, ytrain)

# Predict
preds = ensemble.predict(xtest)


print("Fit data:\n%r" % ensemble.data)
print("Prediction score: %.3f" % accuracy_score(preds, ytest))

Iteration 1, loss = 0.58180038
Validation score: 0.737069
Iteration 2, loss = 0.48609276
Validation score: 0.788793
Iteration 3, loss = 0.42604524
Validation score: 0.778017
Iteration 4, loss = 0.37323962
Validation score: 0.767241
Iteration 5, loss = 0.30374164
Validation score: 0.721983
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 1, loss = 0.62204598
Validation score: 0.724138
Iteration 2, loss = 0.53052635
Validation score: 0.767241
Iteration 3, loss = 0.47464669
Validation score: 0.771552
Iteration 4, loss = 0.44177533
Validation score: 0.788793
Iteration 5, loss = 0.38167239
Validation score: 0.801724
Iteration 6, loss = 0.33114594
Validation score: 0.767241
Iteration 7, loss = 0.28607287
Validation score: 0.784483
Iteration 8, loss = 0.22577970
Validation score: 0.788793
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 1, loss = 0.58180038
Validation score: 0.737069


In [17]:
sklearn.__version__

'0.19.2'

In [0]:
import sklearn

In [0]:
model_name="ensemble_xgb_759"
ts = pd.DataFrame(
{'Unique ID': uid,
 'label': preds
})
ts.to_csv(model_name+".csv",index=False)
from google.colab import files
files.download(model_name+".csv") 

In [11]:
#0.765
from mlens.ensemble import SequentialEnsemble


seed = 2017
np.random.seed(seed)

ensemble = SequentialEnsemble()

# The initial layer is a blended layer, same as a layer in the BlendEnsemble
ensemble.add('blend',
             [LogisticRegression(C=1), RandomForestClassifier(random_state=seed, n_jobs=-1),MLPClassifier((80, 10), early_stopping=True, random_state=seed),
              #GradientBoostingClassifier(n_estimators=100, random_state=seed),
              MLPClassifier((256, 256, 64,8), early_stopping=True, random_state=seed,verbose=True)
             ])

# The second layer is a stacked layer, same as a layer of the SuperLearner
ensemble.add('stack', 
             [LogisticRegression(C=1), RandomForestClassifier(random_state=seed, n_jobs=-1),MLPClassifier((256, 256, 64,8), early_stopping=True, random_state=seed,verbose=True)])

# The third layer is a subsembled layer, same as a layer of the Subsemble
ensemble.add('subsemble', [SVC(), xgb.XGBClassifier(silent=True, n_estimators=120, max_depth=7),GaussianNB()])

# The meta estimator is added as in any other ensemble
ensemble.add_meta(RandomForestClassifier(random_state=seed))

# Fit ensemble
ensemble.fit(xtrain, ytrain)

# Predict
preds = ensemble.predict(xtest)

Iteration 1, loss = 0.58180038
Validation score: 0.737069
Iteration 2, loss = 0.48609276
Validation score: 0.788793
Iteration 3, loss = 0.42604524
Validation score: 0.778017
Iteration 1, loss = 0.62204598
Validation score: 0.724138
Iteration 4, loss = 0.37323962
Validation score: 0.767241
Iteration 2, loss = 0.53052635
Validation score: 0.767241
Iteration 3, loss = 0.47464669
Validation score: 0.771552
Iteration 5, loss = 0.30374164
Validation score: 0.721983
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 4, loss = 0.44177533
Validation score: 0.788793
Iteration 5, loss = 0.38167239
Validation score: 0.801724
Iteration 6, loss = 0.33114594
Validation score: 0.767241
Iteration 7, loss = 0.28607287
Validation score: 0.784483
Iteration 8, loss = 0.22577970
Validation score: 0.788793
Validation score did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 1, loss = 0.57363811
Validation score: 0.646552


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

In [17]:

# Predict
#preds = ensemble.predict(xtest)


print("Fit data:\n%r" % ensemble.data)
print("Prediction score: %.3f" % accuracy_score(preds, ytest))

Fit data:
                                             ft-m  ft-s  pt-m  pt-s
layer-1  gradientboostingclassifier  0    1302.43  0.00  0.18  0.00
layer-1  logisticregression          0      19.00  0.00  0.10  0.00
layer-1  mlpclassifier               0     164.79  0.00  0.67  0.00
layer-1  randomforestclassifier      0       4.15  0.00  0.25  0.00
layer-2  logisticregression          0       0.00  0.00  0.00  0.00
layer-2  mlpclassifier               0       1.04  0.38  0.00  0.00
layer-2  randomforestclassifier      0       0.12  0.00  0.10  0.00
layer-3  gaussiannb                  0       0.00  0.00  0.00  0.00
layer-3  gaussiannb                  1       0.00  0.00  0.00  0.00
layer-3  svc                         0       0.01  0.00  0.02  0.01
layer-3  svc                         1       0.01  0.00  0.01  0.00
layer-3  xgbclassifier               0       0.03  0.00  0.00  0.00
layer-3  xgbclassifier               1       0.03  0.00  0.00  0.00

Prediction score: 0.783


In [16]:
ensemble.model_selection

False